In [1]:
import pandas as pd
import numpy as np
import duckdb

In [2]:
data_folder = "staticNotDeployed/"

In [3]:
scenario_BNZ_path = "simulations_new/BNZ.csv"
scenario_holder_path = "simulations_new/BNZ.csv"

all_scenarios = [scenario_BNZ_path, scenario_holder_path]
scenario_names = ["BNZ", "test"]

## Create pandas tables

In [4]:
dfs = []
for i, scenario in enumerate(all_scenarios):
    df_one_scenario = pd.read_csv(data_folder + scenario)
    df_one_scenario["scenario"] = scenario_names[i]
    dfs.append(df_one_scenario)
df = pd.concat(dfs, axis=0)
del dfs 

In [5]:
# Create total column
df["total (£m)"] = df[ [ f'{i} (£m)' for i in range(2025, 2051)]].sum(axis=1)

In [6]:
# Delete all the columns per hh, only keep total value columns
df = df.drop(columns=[ f'{i} (£/hh)' for i in range(2025, 2051)])

In [7]:
# Convert float64 to float32
df[df.select_dtypes(np.float64).columns] = df.select_dtypes(np.float64).astype(np.float32)

In [8]:
# Rename columns so it does not contain special characters
df.columns = df.columns.str.replace(' (£m)', '')

In [9]:
# Rename columns: replace spaces with underscores
# df.columns = df.columns.str.replace(' ', '_')

In [10]:
df.head()

,Lookup_Value,co_benefit_type,HHs,2025,2026,2027,2028,2029,2030,2031,...,2043,2044,2045,2046,2047,2048,2049,2050,scenario,total
0,N20002754,Air quality,138,0.001385,0.001690,0.002179,0.002650,0.002931,0.002957,0.003004,...,0.003301,0.003274,0.003256,0.003240,0.003194,0.003144,0.003091,0.003035,BNZ,0.078800
1,N20002754,Noise,138,0.000150,0.000145,0.000140,0.000135,0.000775,0.001122,0.001103,...,0.001135,0.001113,0.001176,0.001152,0.001128,0.001105,0.001082,0.001066,BNZ,0.024294
2,N20002754,Congestion,138,0.001159,0.001048,0.000942,0.000893,0.000496,0.000303,0.000562,...,0.004094,0.004525,0.005711,0.006177,0.006654,0.007142,0.007495,0.009113,BNZ,0.073969
3,N20002754,Road repairs,138,0.001225,0.001336,0.001490,0.001566,0.001567,0.001773,0.001761,...,0.001218,0.001152,0.001249,0.001201,0.001212,0.001128,0.001125,0.001155,BNZ,0.035724
4,N20002754,Road safety,138,0.000333,0.000236,0.000191,0.000145,-0.000006,-0.000065,-0.000082,...,0.001388,0.001611,0.002129,0.002397,0.002670,0.002907,0.003154,0.003674,BNZ,0.024474


In [11]:
np.min(df.total)

-8.116110801696777

In [12]:
np.max(df.total)

36.776371002197266

In [13]:
df.dtypes

Lookup_Value        object
co_benefit_type     object
HHs                  int64
2025               float32
2026               float32
2027               float32
2028               float32
2029               float32
2030               float32
2031               float32
2032               float32
2033               float32
2034               float32
2035               float32
2036               float32
2037               float32
2038               float32
2039               float32
2040               float32
2041               float32
2042               float32
2043               float32
2044               float32
2045               float32
2046               float32
2047               float32
2048               float32
2049               float32
2050               float32
scenario            object
total              float32
dtype: object

## Aggregating the time

In [14]:
# Number of years merging
time_step = 5

In [15]:
years = list(range( 2025, 2051 ))
#years

In [16]:
df[["2025", "2026"]]

,2025,2026
0,1.385182e-03,1.690344e-03
1,1.500760e-04,1.450010e-04
2,1.159192e-03,1.048197e-03
3,1.224684e-03,1.336258e-03
4,3.331220e-04,2.359090e-04
...,...,...
557107,2.291570e-05,4.952560e-05
557108,1.056140e-09,-3.498470e-09
557109,1.980370e-07,1.673210e-05
557110,3.025683e-02,4.961723e-02


In [17]:
for i in range(0, len(years) - ( time_step - 1), time_step):
    window_years = [str(year) for year in years[i:i+5]]
    print(window_years)
    window_sum = df[window_years].sum(axis=1)
    df[f'{window_years[0]}_{window_years[-1]}'] = window_sum

['2025', '2026', '2027', '2028', '2029']
['2030', '2031', '2032', '2033', '2034']
['2035', '2036', '2037', '2038', '2039']
['2040', '2041', '2042', '2043', '2044']
['2045', '2046', '2047', '2048', '2049']


In [18]:
df[ ['2025', '2026', '2027', '2028', '2029', '2025_2029'] ]

,2025,2026,2027,2028,2029,2025_2029
0,1.385182e-03,1.690344e-03,2.178994e-03,2.650430e-03,2.930518e-03,1.083547e-02
1,1.500760e-04,1.450010e-04,1.400980e-04,1.353600e-04,7.749510e-04,1.345486e-03
2,1.159192e-03,1.048197e-03,9.423430e-04,8.925020e-04,4.963220e-04,4.538556e-03
3,1.224684e-03,1.336258e-03,1.490412e-03,1.565565e-03,1.567144e-03,7.184063e-03
4,3.331220e-04,2.359090e-04,1.907570e-04,1.449080e-04,-6.340040e-06,8.983559e-04
...,...,...,...,...,...,...
557107,2.291570e-05,4.952560e-05,1.714510e-04,2.048030e-04,2.487060e-04,6.974013e-04
557108,1.056140e-09,-3.498470e-09,5.135090e-09,-4.754620e-09,-1.398150e-09,-3.460010e-09
557109,1.980370e-07,1.673210e-05,1.686910e-05,1.030931e-03,1.709860e-05,1.081829e-03
557110,3.025683e-02,4.961723e-02,6.548754e-02,7.897224e-02,5.226927e-02,2.766031e-01


In [19]:
# Delete single values columns for space
df = df.drop(columns=[str(year) for year in years])

In [21]:
df

,Lookup_Value,co_benefit_type,HHs,scenario,total,2025_2029,2030_2034,2035_2039,2040_2044,2045_2049
0,N20002754,Air quality,138,BNZ,0.078800,1.083547e-02,1.579314e-02,1.662881e-02,1.658240e-02,0.015926
1,N20002754,Noise,138,BNZ,0.024294,1.345486e-03,5.424833e-03,5.019896e-03,5.795703e-03,0.005643
2,N20002754,Congestion,138,BNZ,0.073969,4.538556e-03,2.353550e-03,6.686316e-03,1.809928e-02,0.033179
3,N20002754,Road repairs,138,BNZ,0.035724,7.184063e-03,8.134300e-03,6.970303e-03,6.364777e-03,0.005915
4,N20002754,Road safety,138,BNZ,0.024474,8.983559e-04,-4.291618e-04,1.177130e-03,5.895786e-03,0.013257
...,...,...,...,...,...,...,...,...,...,...
557107,E01006270,Excess cold,634,test,0.067316,6.974013e-04,3.640958e-03,9.472585e-03,1.292229e-02,0.033151
557108,E01006270,Excess heat,634,test,0.000004,-3.460010e-09,3.990457e-08,2.369654e-07,4.301994e-07,0.000002
557109,E01006270,Dampness,634,test,0.042894,1.081829e-03,1.509561e-03,5.385307e-03,5.312776e-03,0.024086
557110,E01006270,Diet change,634,test,0.985556,2.766031e-01,1.806432e-01,1.680467e-01,1.686495e-01,0.169386


In [22]:
df.dtypes

Lookup_Value        object
co_benefit_type     object
HHs                  int64
scenario            object
total              float32
2025_2029          float32
2030_2034          float32
2035_2039          float32
2040_2044          float32
2045_2049          float32
dtype: object

## Export table as parquet file

In [21]:
df.to_parquet('static/database.parquet')

/Users/apister/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


## Create Duckdb instance

The parquet file is currently used in the frontend as the .duckdb file was causing error. This part is therefore not useful for now

In [42]:
DB_FILE_PATH = 'static/database.duckdb'
TABLE_NAME = "cobenefits"

In [43]:
con = duckdb.connect(DB_FILE_PATH)

In [46]:
# Create table and insert data
con.execute(f"DROP TABLE {TABLE_NAME}")

# Create table and insert data
con.execute(f"CREATE TABLE {TABLE_NAME} AS SELECT * FROM df")

# Verify data
result = con.execute(f"SELECT * FROM {TABLE_NAME} LIMIT 5").fetchall()
print("Sample data:")
print(result)

# Get and print schema
schema = con.execute(f"DESCRIBE {TABLE_NAME}").fetchall()
print("\nTable schema:")
for column in schema:
    print(f"{column[0]}: {column[1]}")

print(f"\nDatabase created and saved to: {DB_FILE_PATH}")

Sample data:
[('N20002754', 'Air quality', 138, 0.001385182, 0.001690344, 0.002178994, 0.00265043, 0.002930518, 0.002957105, 0.003004358, 0.003154786, 0.003284162, 0.003392733, 0.003309062, 0.003258978, 0.003316478, 0.003358668, 0.00338563, 0.003360311, 0.003329033, 0.00331858, 0.003300776, 0.003273695, 0.003256345, 0.003240189, 0.00319424, 0.003143862, 0.003091112, 0.00303487, 'BNZ', 0.078800441), ('N20002754', 'Noise', 138, 0.000150076, 0.000145001, 0.000140098, 0.00013536, 0.000774951, 0.001122295, 0.001103271, 0.001084611, 0.001066307, 0.001048349, 0.001038151, 0.001020742, 0.001003659, 0.000986897, 0.000970447, 0.001206611, 0.001182306, 0.001158578, 0.001135413, 0.001112795, 0.001176065, 0.001151699, 0.00112793, 0.00110474, 0.001082113, 0.00106603, 'BNZ', 0.024294494999999996), ('N20002754', 'Congestion', 138, 0.001159192, 0.001048197, 0.000942343, 0.000892502, 0.000496322, 0.000302559, 0.000561585, 0.000513852, 0.000433189, 0.000542365, 0.001034313, 0.001105454, 0.001294352, 0.00